In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.sparse.linalg import eigsh
#from pretrainedModel import pretrainedModel
from tensorflow import keras
from PIL import Image
from sklearn.preprocessing import StandardScaler
import torch
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
import time
import warnings
from sklearn.cluster import KMeans
import sys
import os

from copy import deepcopy

In [2]:
import random
import torch.nn as nn
import numpy as np
import torch
import math
import torch.optim as optim
from torch.utils.data import DataLoader

root = '../../'
bits = 12

In [3]:
torch.tensor( np.load( root + r"Features/HPO og Validering/CIFAR/y_hpo_CIfar.npy" ) )

tensor([6, 3, 2,  ..., 1, 5, 3])

In [4]:
def get_cifar():
    """    
    X_train, y_train, X_val, y_val
    """
    
    X_train = torch.tensor( np.load( root + r"Features/HPO og Validering/CIFAR/X_hpo_Cifar.npy" ) )
    y_train = torch.tensor( np.load( root + r"Features/HPO og Validering/CIFAR/y_hpo_CIfar.npy" ) )

    label_amnt = len(y_train.unique())
    print(y_train)
    y_train = torch.nn.functional.one_hot(y_train, label_amnt)

    X_val = torch.tensor( np.load( root + r"Features/HPO og Validering/CIFAR/X_val_Cifar.npy" ) )
    y_val = torch.tensor( np.load( root + r"Features/HPO og Validering/CIFAR/y_val_Cifar.npy" ) )

    label_amnt = len(y_val.unique())
    y_val = torch.nn.functional.one_hot(y_val, label_amnt)

    return X_train, y_train, X_val, y_val

get_cifar()

tensor([6, 3, 2,  ..., 1, 5, 3])


(tensor([[-0.2165,  0.4864, -0.6618,  ..., -1.0460, -0.9252, -1.1836],
         [-0.0562, -1.9006, -3.1185,  ..., -1.3443,  1.2088, -1.9138],
         [-1.7397, -0.6687, -1.1167,  ..., -1.7901, -1.4354, -1.4159],
         ...,
         [-0.5435, -1.0836, -2.4497,  ..., -1.0178, -0.8090,  0.3407],
         [-1.4420, -1.2296, -1.2801,  ..., -1.7022, -0.4764, -1.6929],
         [-0.3117, -1.4599, -2.6039,  ...,  0.1516,  0.1330,  0.6513]]),
 tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 1,  ..., 0, 0, 0],
         ...,
         [0, 1, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 tensor([[-1.9692, -1.5296, -2.8161,  ..., -1.7515,  1.1944,  0.8192],
         [ 1.3917, -0.5071, -2.9970,  ..., -0.7656, -1.0312,  0.9559],
         [ 1.8041, -1.3244, -3.4150,  ..., -0.0814, -0.7052,  0.7900],
         ...,
         [-0.2618, -1.9799, -1.4752,  ..., -1.4723, -0.2433,  2.1289],
         [-1.5827, -0.9795, -1.715

In [5]:
def get_imgNet():
    """    
    X_train, y_train, X_val, y_val
    """
    
    X_train = torch.tensor( np.load( root + r"Features\HPO og Validering\Imagenet\X_hpo_Img.npy" ) )
    y_train = torch.tensor( np.load( root + r"Features/HPO og Validering/Imagenet/y_hpo_Img.npy" ) )

    y_train = torch.tensor(y_train, dtype=torch.long)

    y_train = torch.nn.functional.one_hot(y_train)

    X_val = torch.tensor( np.load( root + r"Features/HPO og Validering/Imagenet/X_val_Img.npy" ) )
    y_val = torch.tensor( np.load( root + r"Features/HPO og Validering/Imagenet/y_val_Img.npy" ) )


    y_val = torch.tensor(y_val, dtype=torch.long)

    y_val = torch.nn.functional.one_hot(y_val)

    return X_train, y_train, X_val, y_val

In [6]:
def get_nus():
    """    
    X_train, y_train, X_val, y_val
    """
    
    X_train = torch.tensor( np.load( root + r"Features\HPO og Validering\Nus_Wide\X_hpo_Nus.npy" ) )
    y_train = torch.tensor( np.load( root + r"Features/HPO og Validering/Nus_Wide/y_hpo_Nus.npy" ) )

    y_train = torch.tensor(y_train, dtype=torch.long)

    X_val = torch.tensor( np.load( root + r"Features/HPO og Validering/Nus_Wide/X_val_Nus.npy" ) )
    y_val = torch.tensor( np.load( root + r"Features/HPO og Validering/Nus_Wide/y_val_Nus.npy" ) )


    y_val = torch.tensor(y_val, dtype=torch.long)

    return X_train, y_train, X_val, y_val

get_nus()

C:\Users\thors\AppData\Local\Temp\ipykernel_14440\1840474892.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.long)
C:\Users\thors\AppData\Local\Temp\ipykernel_14440\1840474892.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_val = torch.tensor(y_val, dtype=torch.long)


(tensor([[-1.5463, -1.9827, -2.6451,  ..., -2.0036,  1.2603, -1.1891],
         [-1.2541, -1.0105, -0.6011,  ..., -2.4619, -1.5300, -1.7605],
         [-1.1112,  2.4095,  0.3236,  ...,  0.7627, -1.5408, -0.6779],
         ...,
         [-0.5131,  0.2285,  1.8414,  ..., -2.4168, -0.8139, -1.6194],
         [-2.2052, -1.9449, -2.5375,  ..., -3.7375, -0.4675, -0.1241],
         [-1.1212, -0.7011, -2.9082,  ..., -1.5942,  0.0986, -0.8006]]),
 tensor([[0, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0],
         [0, 0, 1,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1]]),
 tensor([[-0.7948, -1.8355, -1.4885,  ..., -0.6794,  0.6961,  0.8588],
         [-1.1886, -1.8034, -3.9318,  ..., -2.2932, -1.0354,  0.5069],
         [-2.5658, -2.8377, -2.6987,  ..., -2.1929, -1.4571,  0.2324],
         ...,
         [-2.5087, -1.9090, -1.1950,  ..., -1.9734, -1.3159,  0.0562],
         [-1.6681,  0.0652, -0.660

In [7]:
def get_dataloader(file, batchSize = 100):
    """
    "cifar"
    "imgnet"
    "nus"
    """
    file = file.lower()
    legal_files = ["cifar", "imgnet", "nus"]
    if file not in legal_files:
        raise ValueError(f'The given file name was "{file}", expected from {legal_files}')


    # == LOAD IN THE DATA ==
    if file == "cifar":
        X_train, y_train, _, _ = get_cifar()
    elif file == "imgnet":
        X_train, y_train, _, _ = get_imgNet()
    elif file == "nus":
        X_train, y_train, _, _ = get_nus()

    # == MAKE DATA LOADER ==
    train_data = []
    for i in range(len(X_train)):
        train_data.append([X_train[i], y_train[i]])
    dataloader = DataLoader(train_data, batch_size=batchSize, shuffle=True)


    return dataloader

for i,batch  in enumerate(get_dataloader("nus")):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


C:\Users\thors\AppData\Local\Temp\ipykernel_14440\1840474892.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.long)
C:\Users\thors\AppData\Local\Temp\ipykernel_14440\1840474892.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_val = torch.tensor(y_val, dtype=torch.long)


In [8]:
class DTSHLoss(torch.nn.Module):
    def __init__(self):
        super(DTSHLoss, self).__init__()

    def forward(self, u, y, LAMBDA=1, ALPHA=1):
        #LAMBDA = 1
        #ALPHA  = 1

        inner_product = u @ u.t()   # Similarity Matrix
        s = y @ y.t() > 0           # A matrix that show if the two idexes are the same or not
        count = 0

        loss1 = 0
        for row in range(s.shape[0]):
            # if has positive pairs and negative pairs
            if s[row].sum() != 0 and (~s[row]).sum() != 0:
                count += 1
                theta_positive = inner_product[row][s[row] == 1]                
                theta_negative = inner_product[row][s[row] == 0]

                triple = (theta_positive.unsqueeze(1) - theta_negative.unsqueeze(0) - ALPHA ).clamp(min=-100,max=50)
                loss1 += -(triple - torch.log(1 + torch.exp(triple))).mean()

        if count != 0:
            loss1 = loss1 / count
        else:
            loss1 = 0

        loss2 = LAMBDA * (u - u.sign()).pow(2).mean()

        return loss1 + loss2

In [9]:
from tqdm import tqdm

def one_hot_encode(a):
    b = np.zeros((a.size, a.max() + 1))
    b[np.arange(a.size), a] = 1
    return b

def mean_average_precision(test_hashes, training_hashes, test_labels, training_labels):
    aps = []
    if len(training_labels.shape) == 1:
        training_labels = one_hot_encode(training_labels)
        test_labels = one_hot_encode(test_labels)
    for i, test_hash in enumerate(tqdm(test_hashes)):
        label = test_labels[i]
        distances = np.abs(training_hashes - test_hashes[i]).sum(axis=1)
        tp = np.where((training_labels*label).sum(axis=1)>0, 1, 0)
        hash_df = pd.DataFrame({"distances":distances, "tp":tp}).reset_index()
        hash_df = hash_df.sort_values(["distances", "index"]).reset_index(drop=True)
        hash_df = hash_df.drop(["index", "distances"], axis=1).reset_index()
        hash_df = hash_df[hash_df["tp"]==1]
        hash_df["tp"] = hash_df["tp"].cumsum()
        hash_df["index"] = hash_df["index"] +1 
        precision = np.array(hash_df["tp"]) / np.array(hash_df["index"])
        ap = precision.mean()
        aps.append(ap)
    
    return np.array(aps).mean()

In [10]:
def earlyStop(LossList, n = 10):
    bestVal = min(LossList)

    bestVal_i = LossList.index(bestVal)

    if bestVal_i < len(LossList) - n: return True

    

In [11]:
def res(X, model):
    results =  model(X)
    results = results.detach().numpy()

    results = (results > 0.5).astype(int) 

    return results

In [12]:
class DTSHLoss(torch.nn.Module):
    def __init__(self):
        super(DTSHLoss, self).__init__()

    def forward(self, u, y, LAMBDA=1, ALPHA=1):
        #LAMBDA = 1
        #ALPHA  = 1

        inner_product = torch.cdist(u, u, p=2) 
        s = y @ y.t() > 0           # A matrix that show if the two idexes are the same or not

        loss1 = torch.tensor(0.0, requires_grad=True) + 0
        for row in range(s.shape[0]):
            # if has positive pairs and negative pairs
            if s[row].sum() != 0 and (~s[row]).sum() != 0:
                
                theta_negative = inner_product[row][s[row] == 0]
                
                theta_positive = inner_product[row][s[row] == 1]
                theta_positive = theta_positive[theta_positive != 0] # remove the anchor

                for p in theta_positive: 
                    n_i = torch.logical_and( (p < theta_negative), (theta_negative < p + ALPHA) )
                    
                    if sum(n_i) != 0:
                        n = torch.min( theta_negative[n_i] )

                        loss1 += (p - n + ALPHA).clamp(min=0)
        
        #if loss1 == 0: return torch.Tensor(0,  requires_grad=True)

        return loss1

In [13]:
"""
lr           = 1e-6
weight_decay = 1e-6

model = nn.Sequential(  nn.Linear(4096,256),
                            nn.ReLU(),
                            nn.Linear(256, bits),
                            nn.Sigmoid()
                            )
criterion = DTSHLoss()
optimizer = optim.RMSprop(model.parameters(), lr=lr , weight_decay=weight_decay)
"""

'\nlr           = 1e-6\nweight_decay = 1e-6\n\nmodel = nn.Sequential(  nn.Linear(4096,256),\n                            nn.ReLU(),\n                            nn.Linear(256, bits),\n                            nn.Sigmoid()\n                            )\ncriterion = DTSHLoss()\noptimizer = optim.RMSprop(model.parameters(), lr=lr , weight_decay=weight_decay)\n'

In [14]:
"""
for i in range(10):
    
    dataloader = get_dataloader("cifar", batchSize=50)
    L = []

    for j,batch  in enumerate(dataloader):
        

        X_batch = batch[0]
        y_batch = batch[1]

        u = model(X_batch)



        loss = criterion(u, y_batch.float(), ALPHA=3)
        L.append(loss)

        loss.backward()
        optimizer.step()
    print( sum(L) / len(L) )
"""

'\nfor i in range(10):\n    \n    dataloader = get_dataloader("cifar", batchSize=50)\n    L = []\n\n    for j,batch  in enumerate(dataloader):\n        \n\n        X_batch = batch[0]\n        y_batch = batch[1]\n\n        u = model(X_batch)\n\n\n\n        loss = criterion(u, y_batch.float(), ALPHA=3)\n        L.append(loss)\n\n        loss.backward()\n        optimizer.step()\n    print( sum(L) / len(L) )\n'

In [15]:
def HPO(HP):
    time_start = time.time()

    data = {}

    # LAMBDA=1, 
    # ALPHA=1
    # lr=1e-5
    # weight_decay=1e-5
    # bits = 16

    #ALPHA=  HP["alpha"]
    lr=     HP["lr"]
    weight_decay= HP["wd"]
    bits = HP["bits"]


    model = nn.Sequential(  nn.Linear(4096,256),
                            nn.ReLU(),
                            nn.Linear(256, bits),
                            )

    criterion = DTSHLoss()
    optimizer = optim.RMSprop(model.parameters(), lr=lr , weight_decay=weight_decay)

    dataloader = get_dataloader("nus", batchSize=50)
    historical_lostList = []
    for i in range(1000):
        loss_list = []
        for j,batch  in enumerate(dataloader):
            X_batch = batch[0]
            y_batch = batch[1]

            optimizer.zero_grad()

            u = model(X_batch)
            loss = criterion(u, y_batch.float(), ALPHA=HP["alpha"] )
            loss.backward()
            optimizer.step()

            loss_list.append( float(loss) )
        
        
        mean_loss = sum(loss_list) / len(loss_list)
        if i % 10 == 1:
            print(i, mean_loss)
        historical_lostList.append(mean_loss)

        if earlyStop(historical_lostList, n = 10): 
            print(i, mean_loss)
            print("Early Stop!!!")
            data["earlyStop"] = True
            break
        
        if time.time() - time_start > 60*30: 
            data["time stopage"] = True
            break

    
    # === EVALUATE ===
    #X_train, y_train, X_val, y_val = get_cifar()
    X_train, y_train, X_val, y_val = get_nus()

    
    #data["hp"] = HP
    data["loss"] = historical_lostList
    data["map"] = mean_average_precision(test_hashes=res(X_val, model), training_hashes=res(X_train, model), test_labels=y_val, training_labels=y_train)
    return data

In [16]:
from copy import deepcopy
import os
import json
import datetime
import time
from joblib import Parallel, delayed
import random

def _get_combos(D):
    l = list(D)
    curr_i = dict(zip(l,[0]*len(l)))
    combos = []

    while True:
        for i in range(len(l)):
            val = l[i]

            if curr_i[val] > (len(D[val])-1):
                curr_i[val] = 0
                
                if i+1 != len(l):
                    curr_i[ l[i+1] ] += 1                    
                else:
                    return combos                    

        combo = deepcopy(D)
        for key in curr_i:
            list_ = combo[key]
            index = curr_i[key] 
             
            combo[key] = list_[index]

        combos.append( combo )
        curr_i[l[0]] += 1

def save_dict(d, path, indentifier=""):
    # === CLEAN PATH ===
    path = path.replace("\\", "/") # Ensures that it's always "/" and not "\"

    if path[-1] != "/": # Ensures that path ends with "/"
        path += "/"

    # === DEFINE FOLDER ===
    folder_path = f"{path}{os.environ['COMPUTERNAME']}"

    if not os.path.exists(folder_path): # Makes the Path if it doesn't exist
        os.makedirs(folder_path)

    # === DEFINE FILE NAME/PATH ===
    now = str(datetime.datetime.now())
    now = now[:now.index(".")].replace(" ", "_").replace(":", ";")

    if indentifier == "":
        filePath = f"{folder_path}/{now}.json"
    else:
        filePath = f"{folder_path}/{now}_{indentifier}.json"

    # === SAVE FILE ===
    with open(filePath, "w") as fp:
        json.dump(d , fp)

def read_folder(folder_path):
    """
    Opens all folder inside "folder_path" and reads the contents of thoose folders.
    """

    data_list = []

    for folder in os.listdir(folder_path):
        folderPath = folder_path+"/"+folder
        
        for file in os.listdir(folderPath):
            filePath = f"{folderPath}/{file}"
            with open( filePath ) as json_file:
                data = json.load(json_file)
                data["filePath"] = filePath

                data_list.append( data )
    
    return data_list

def work(task, path, func, n_jobs=10, taskName="task", shuffle_tasks=True, verbose=1): 
    # === CLEAN PATH + MAKE FOLDeR ===
    path = path.replace("\\", "/") # Ensures that it's always "/" and not "\"
    if path[-1] != "/": # Ensures that path ends with "/"
        path += "/"
    folder_path = f"{path}{os.environ['COMPUTERNAME']}"
    if not os.path.exists(folder_path): # Makes the Path if it doesn't exist
        os.makedirs(folder_path)
    
    # === DEFINE COMBOS / TASKS ===
    tasks = _get_combos( task )
    
    done_tasks =[ d[taskName] for d in read_folder( path ) ]

    remaining_tasks = [t for t in tasks if t not in done_tasks]

    if shuffle_tasks: random.shuffle(remaining_tasks)

    # === WRAPPER FOR "FUNC" ===
    def moddified_func(t, i):               
        start_time = time.time()

        result_dict = {}
        
        result_dict[taskName] = t # the task

        result_dict["result"] = func(t) # the result of doing the task
        
        end_time = time.time()
        time_spend = end_time - start_time
        result_dict["time"] = time_spend

        save_dict(result_dict, path, indentifier=i)

        return result_dict

    # === PARALLEL PROCCESING ===
    func_results =  Parallel(n_jobs=n_jobs, verbose=verbose)( [ delayed(moddified_func)(t, i) for i, t in enumerate(remaining_tasks) ] )

    return func_results

In [17]:
HP = {
    "alpha"  : [1,3,5],
    "lr"     : [1e-4 ,1e-5, 1e-6],
    "wd"     : [1e-4 ,1e-5, 1e-6],
    "bits"   : [12, 24, 32, 48]
}


work(HP, root+r"Results\HPO\TripletAgain\Nus", HPO)

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed: 90.9min
[Parallel(n_jobs=10)]: Done 108 out of 108 | elapsed: 332.0min finished


[{'task': {'alpha': 5, 'lr': 0.0001, 'wd': 1e-06, 'bits': 48},
  'result': {'time stopage': True,
   'loss': [2217.105144840656,
    1777.8642989054765,
    1643.329277416267,
    1569.8864727964497,
    1509.3885576606976,
    1442.043719905438,
    1428.7052213461093,
    1391.2916477316678,
    1379.2849725402227,
    1332.679251189279,
    1314.6502748999264,
    1298.0346589041228,
    1302.4076121113087,
    1280.0053626334313,
    1289.103638299621,
    1262.5610835009281,
    1246.6747938099475,
    1242.8436744614403,
    1236.976953487585,
    1227.6675046410892,
    1223.3486624236155,
    1194.9144710125308,
    1213.0088271339341,
    1184.590913980314,
    1173.5084143912438,
    1197.6756881864944,
    1165.6772436765161,
    1178.7185874410195,
    1150.277233765857,
    1158.4864350876005,
    1143.2883101359453,
    1155.997341042698,
    1140.227239325495,
    1116.5441967048268,
    1111.3329328782488,
    1107.8241051400062,
    1067.2773926989867,
    1061.3750145